In [8]:
import types
import tempfile
import keras.models
import mne as mn
from sklearn.metrics import mean_squared_error

In [9]:
def make_keras_picklable():
    def __getstate__(self):
        model_str = ""
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            keras.models.save_model(self, fd.name, overwrite=True)
            model_str = fd.read()
        d = { 'model_str': model_str }
        return d

    def __setstate__(self, state):
        with tempfile.NamedTemporaryFile(suffix='.hdf5', delete=True) as fd:
            fd.write(state['model_str'])
            fd.flush()
            model = keras.models.load_model(fd.name)
        self.__dict__ = model.__dict__


    cls = keras.models.Model
    cls.__getstate__ = __getstate__
    cls.__setstate__ = __setstate__

Сохранение данных с помощью pickle. encoder и decoder это ваши обученные модели

In [ ]:
make_keras_picklable()
pickle.dump(encoder, open("encoder.p", "wb"))
pickle.dump(decoder, open("decoder.p", "wb"))

Загрузка данных и вычисление score. something - это ваш код, который как-то возможно меняет данные, например транспонирует или добавляет новые координаты, главное чтобы после encodera, данные были в формате $(n \times m)$, коэффицент будет вычисляться по параметру $m$

In [ ]:
Your code on python to transform (n,61) to (n, 64)
raw = mn.io.read_raw_brainvision("./data_test")
raw_data = raw.get_data()
x_test = something(raw_data)

Запуск обученных моделей. Чем меньше score тем лучше

In [ ]:
load_encoder = pickle.load(open("encoder.p", "rb"))
load_decoder = pickle.load(open("decoder.p", "rb"))
x_reduce = load_encoder.predict(x_test)
alpha = x_reduce.shape[1] / 64
x_pred = load_decoder.predict(x_reduce)
mse = mean_squared_error(x_train, x_test)
score = (1 + mse) * alpha
if (alpha > 0.9): 
    score = 100000 #infty
print(score)